# 將清洗後的資料，從Python導入儲存至MySQL建置資料庫

In [1]:
import pymysql

#設定連線資訊
host = 'localhost'
port = 3306
user = 'root'
passwd = 'xxxxx'
charset = 'utf8mb4'

#建立連線
conn = pymysql.connect(host=host,port=port,user=user,passwd=passwd,charset=charset,local_infile=True)
print('Successfully connected')

Successfully connected


In [2]:
#建立游標
cursor = conn.cursor()

#建立新資料庫
cursor.execute("CREATE DATABASE IF NOT EXISTS Final_Group_Project_DB;")

1

In [3]:
#切換新建立資料庫 Final_Group_Project_DB
sql ="""
USE Final_Group_Project_DB;
"""
cursor.execute(sql)

0

In [4]:
#建立 product TABLE
sql ="""
CREATE TABLE product (
            PRODUCT_ID INT NOT NULL,
            MANUFACTURER INT,
            DEPARTMENT VARCHAR(20),
            BRAND VARCHAR(20),
            COMMODITY_DESC VARCHAR(40),
            SUB_COMMODITY_DESC VARCHAR(40),
            CURR_SIZE_OF_PRODUCT VARCHAR(20),
            
            CONSTRAINT PRODUCKT_ID_PK PRIMARY KEY (PRODUCT_ID)

)ENGINE = INNODB;
"""
cursor.execute(sql)

0

In [5]:
#匯入product.csv檔案到資料庫
sql ="""
LOAD DATA LOCAL INFILE "./data/product.csv" 
INTO TABLE product
FIELDS TERMINATED BY ',' 
ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 ROWS
(PRODUCT_ID,MANUFACTURER,DEPARTMENT,BRAND,COMMODITY_DESC,SUB_COMMODITY_DESC,CURR_SIZE_OF_PRODUCT);
"""
cursor.execute(sql)

92353

In [6]:
#交易確認送出
conn.commit()

In [7]:
#建立 hh_demographic TABLE 
sql ="""
CREATE TABLE hh_demographic (
            AGE_DESC VARCHAR(10),
            MARITAL_STATUS_CODE VARCHAR(5),
            INCOME_DESC VARCHAR(20),
            HOMEOWNER_DESC VARCHAR(20),
            HH_COMP_DESC VARCHAR(20),
            HOUSEHOLD_SIZE_DESC VARCHAR(5),
            KID_CATEGORY_DESC VARCHAR(20),
            household_key INT NOT NULL,
            
            CONSTRAINT household_key_PK PRIMARY KEY (household_key)
            
)ENGINE = INNODB;
"""
cursor.execute(sql)

0

In [8]:
#匯入hh_demographic_filling_missing_data.csv檔案到資料庫
sql ="""
LOAD DATA LOCAL INFILE "./data/hh_demographic_filling_missing_data.csv" 
INTO TABLE hh_demographic
FIELDS TERMINATED BY ',' 
ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 ROWS
(AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE_DESC,KID_CATEGORY_DESC,household_key);
"""
cursor.execute(sql)

2500

In [9]:
#交易確認送出
conn.commit()

In [10]:
#建立 campaign_desc TABLE
sql ="""
CREATE TABLE campaign_desc (
            DESCRIPTION VARCHAR(10),
            CAMPAIGN SMALLINT NOT NULL,
            START_DAY SMALLINT,
            END_DAY SMALLINT,
            
            CONSTRAINT CAMPAIGN_PK PRIMARY KEY (CAMPAIGN)
            
)ENGINE = INNODB;
"""
cursor.execute(sql)

0

In [11]:
# 匯入campaign_desc.csv檔案到資料庫
sql ="""
LOAD DATA LOCAL INFILE "./data/campaign_desc.csv" 
INTO TABLE campaign_desc
FIELDS TERMINATED BY ',' 
ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 ROWS
(DESCRIPTION,CAMPAIGN,START_DAY,END_DAY);
"""
cursor.execute(sql)

30

In [12]:
#交易確認送出
conn.commit()

In [13]:
#建立 campaign_table TABLE
sql ="""
CREATE TABLE campaign_table (
            DESCRIPTION VARCHAR(10),
            household_key INT,
            CAMPAIGN SMALLINT,
            
            CONSTRAINT household_key_FK FOREIGN KEY (household_key) REFERENCES hh_demographic (household_key),
            CONSTRAINT CAMPAIGN_FK FOREIGN KEY (CAMPAIGN) REFERENCES campaign_desc(CAMPAIGN)
            
)ENGINE = INNODB;
"""
cursor.execute(sql)

0

In [14]:
#匯入campaign_table.csv檔案到資料庫
sql ="""
LOAD DATA LOCAL INFILE "./data/campaign_table.csv" 
INTO TABLE campaign_table
FIELDS TERMINATED BY ',' 
ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 ROWS
(DESCRIPTION,household_key,CAMPAIGN);
"""
cursor.execute(sql)

7208

In [15]:
#交易確認送出
conn.commit()

In [16]:
#建立 coupon TABLE
sql ="""
CREATE TABLE coupon (
            COUPON_UPC BIGINT,
            PRODUCT_ID INT,
            CAMPAIGN SMALLINT,
            
            CONSTRAINT PRODUCT_ID_FK_1 FOREIGN KEY (PRODUCT_ID) REFERENCES product (PRODUCT_ID),
            CONSTRAINT CAMPAIGN_FK_1 FOREIGN KEY (CAMPAIGN) REFERENCES campaign_desc (CAMPAIGN)
            
)ENGINE = INNODB;
"""
cursor.execute(sql)

0

In [17]:
#匯入coupon.csv檔案到資料庫
sql ="""
LOAD DATA LOCAL INFILE "./data/coupon.csv" 
INTO TABLE coupon
FIELDS TERMINATED BY ',' 
ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 ROWS
(COUPON_UPC,PRODUCT_ID,CAMPAIGN);
"""
cursor.execute(sql)

124548

In [18]:
#交易確認送出
conn.commit()

In [19]:
#建立 transaction_data TABLE
sql ="""
CREATE TABLE transaction_data (
            household_key INT,
            BASKET_ID BIGINT,
            DAY SMALLINT,
            PRODUCT_ID INT NOT NULL,
            QUANTITY INT,
            SALES_VALUE FLOAT,
            STORE_ID INT,
            RETAIL_DISC FLOAT,
            TRANS_TIME SMALLINT,
            WEEK_NO SMALLINT,
            COUPON_DISC FLOAT,
            COUPON_MATCH_DISC FLOAT,
            
            CONSTRAINT household_key_FK_1 FOREIGN KEY (household_key) REFERENCES hh_demographic (household_key),
            CONSTRAINT PRODUCT_ID_FK_2 FOREIGN KEY (PRODUCT_ID) REFERENCES product (PRODUCT_ID)
            
)ENGINE = INNODB;
"""
cursor.execute(sql)


0

In [20]:
#匯入transaction_data.csv檔案到資料庫
sql ="""
LOAD DATA LOCAL INFILE "./data/transaction_data.csv" 
INTO TABLE transaction_data
FIELDS TERMINATED BY ',' 
ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 ROWS
(household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC);
"""
cursor.execute(sql)

2595732

In [21]:
#交易確認送出
conn.commit()

In [22]:
#建立 coupon_redempt TABLE
sql ="""
CREATE TABLE coupon_redempt (
            household_key INT,
            DAY SMALLINT,
            COUPON_UPC BIGINT,
            CAMPAIGN SMALLINT,
            
            CONSTRAINT household_key_FK_2 FOREIGN KEY (household_key) REFERENCES hh_demographic (household_key),
            CONSTRAINT CAMPAIGN_FK_2 FOREIGN KEY (CAMPAIGN) REFERENCES campaign_desc(CAMPAIGN)
            
)ENGINE = INNODB;
"""
cursor.execute(sql)

0

In [23]:
#匯入coupon_redempt.csv檔案到資料庫
sql ="""
LOAD DATA LOCAL INFILE "./data/coupon_redempt.csv" 
INTO TABLE coupon_redempt
FIELDS TERMINATED BY ',' 
ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 ROWS
(household_key,DAY,COUPON_UPC,CAMPAIGN);
"""
cursor.execute(sql)

2318

In [24]:
#交易確認送出
conn.commit()

In [25]:
#建立 causal_data TABLE
sql ="""
CREATE TABLE causal_data (
            PRODUCT_ID INT NOT NULL,
            STORE_ID INT NOT NULL,
            WEEK_NO SMALLINT,
            display VARCHAR(5),
            mailer VARCHAR(5),
            
            CONSTRAINT PRODUCT_ID_FK FOREIGN KEY (PRODUCT_ID) REFERENCES product (PRODUCT_ID)
            
)ENGINE = INNODB;
"""
cursor.execute(sql)

0

In [26]:
#匯入causal_data.csv檔案到資料庫
for i in range(0,37):
    sql ="""
    LOAD DATA LOCAL INFILE "./data/causaldata_splited/causal_data_%s.csv"
    INTO TABLE causal_data
    FIELDS TERMINATED BY ',' 
    ENCLOSED BY '"'
    LINES TERMINATED BY '\n'
    IGNORE 1 ROWS
    (PRODUCT_ID,STORE_ID,WEEK_NO,display,mailer);"""  % i

    cursor.execute(sql)
    #交易確認送出
    conn.commit()


In [27]:
# sql ="SELECT * FROM campaign_desc;"
# cursor.execute(sql)

In [28]:
# data = cursor.fetchall()
# data

In [29]:
#關閉游標及連線
cursor.close()
conn.close()